# PREPARING DATA

In [32]:
import pandas as pd
import glob
import os
import numpy as np

# Path ke folder train
folder_path = r"D:\Data Sains\dataset\comodity-price-prediction-penyisihan-arkavidia-9\iris n\Harga Bahan Pangan\train"
output_file = "merged_commodities.csv"

# Ambil semua file CSV dalam folder train
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# Dictionary untuk menyimpan rentang harga tiap komoditas
price_ranges = {}

# Identifikasi rentang harga dari setiap komoditas
for file in csv_files:
    # Ambil nama komoditas dari nama file
    commodity_name = os.path.splitext(os.path.basename(file))[0]
    
    # Baca CSV
    df = pd.read_csv(file)
    
    # Konversi semua kolom kecuali tanggal ke numerik
    df.iloc[:, 1:] = df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
    
    # Ambil hanya nilai harga tanpa tanggal
    harga_values = df.iloc[:, 1:].values.flatten()
    
    # Buang NaN agar tidak mengganggu perhitungan
    harga_values = harga_values[~np.isnan(harga_values)]
    
    # Hitung rentang harga berdasarkan metode IQR (Interquartile Range)
    Q1 = np.percentile(harga_values, 25)
    Q3 = np.percentile(harga_values, 75)
    IQR = Q3 - Q1
    min_price = max(0, Q1 - 1.5 * IQR)  # Harga tidak mungkin negatif
    max_price = Q3 + 1.5 * IQR
    
    # Simpan rentang harga ke dictionary
    price_ranges[commodity_name] = (min_price, max_price)

# List untuk menyimpan DataFrame hasil transformasi
df_list = []

# Proses ulang setiap file CSV untuk menggabungkan data
for file in csv_files:
    # Ambil nama komoditas dari nama file
    commodity_name = os.path.splitext(os.path.basename(file))[0]
    
    # Baca CSV
    df = pd.read_csv(file)
    
    # Unpivot (melt) data
    df_melted = df.melt(id_vars=["Date"], var_name="provinsi", value_name="harga")
    
    # Tambahkan kolom komoditas
    df_melted["komoditas"] = commodity_name
    
    # Konversi harga ke numerik untuk memastikan konsistensi
    df_melted["harga"] = pd.to_numeric(df_melted["harga"], errors='coerce')
    
    # Handle outlier menggunakan rentang harga yang sudah dihitung
    min_price, max_price = price_ranges[commodity_name]
    df_melted.loc[(df_melted["harga"] < min_price) | (df_melted["harga"] > max_price), "harga"] = np.nan
    
    # Tambahkan kolom id
    df_melted["id"] = df_melted["komoditas"] + "/" + df_melted["provinsi"] + "/" + df_melted["Date"]
    
    # Simpan hasil
    df_list.append(df_melted)

# Gabungkan semua DataFrame
final_df = pd.concat(df_list, ignore_index=True)

# Simpan ke CSV
final_df.to_csv(output_file, index=False)

print("File gabungan berhasil dibuat:", output_file)


# Path folder test
folder_path_2 = r"D:\Data Sains\dataset\comodity-price-prediction-penyisihan-arkavidia-9\iris n\Harga Bahan Pangan\test"
output_file_2 = "test_clean.csv"

# Ambil semua file CSV dalam folder test
csv_files = glob.glob(os.path.join(folder_path_2, "*.csv"))

# List untuk menyimpan DataFrame hasil transformasi
df_list = []

# Proses setiap file CSV dan gabungkan
for file in csv_files:
    # Ambil nama komoditas dari nama file
    commodity_name = os.path.splitext(os.path.basename(file))[0]
    
    # Baca CSV
    df = pd.read_csv(file)
    
    # Unpivot (melt) data
    df_melted = df.melt(id_vars=["Date"], var_name="provinsi", value_name="harga")
    
    # Tambahkan kolom komoditas
    df_melted["komoditas"] = commodity_name
    
    # Tambahkan kolom id
    df_melted["id"] = df_melted["komoditas"] + "/" + df_melted["provinsi"] + "/" + df_melted["Date"]
    
    # Simpan hasil
    df_list.append(df_melted)

# Gabungkan semua DataFrame
final_df = pd.concat(df_list, ignore_index=True)

final_df["Date"] = pd.to_datetime(final_df["Date"], errors='coerce')

# Simpan ke CSV
final_df.to_csv(output_file_2, index=False)

print("File gabungan berhasil dibuat:", output_file_2)

submission = pd.read_csv(r"D:\Data Sains\dataset\comodity-price-prediction-penyisihan-arkavidia-9\iris n\Harga Bahan Pangan\sample_submission.csv")

File gabungan berhasil dibuat: merged_commodities.csv
File gabungan berhasil dibuat: test_clean.csv


In [33]:
# Baca File Merge_commodities dan test
train = pd.read_csv("merged_commodities.csv")
test = pd.read_csv("test_clean.csv")
test = test.drop(columns=["harga"], errors="ignore")
test['Date'] = pd.to_datetime(test['Date'], format='%Y-%m-%d', errors='coerce')

# fill harga missing with interpolasi order 1
train["harga"] = (
    train.groupby(["komoditas", "provinsi"], group_keys=False)["harga"]
    .apply(lambda x: x.interpolate())
    .reset_index(drop=True)
)

# fill rest of missing with bfill
train["harga"] = train.groupby(["komoditas", "provinsi"])["harga"].fillna(method="ffill").fillna(method="bfill")

# save to train_clean.csv
train.to_csv("train_clean.csv")

train['Date'] = pd.to_datetime(train['Date'], format='%Y-%m-%d', errors='coerce')

# MODEL

In [34]:

import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import warnings
import os

# Abaikan warning dari ARIMA
warnings.filterwarnings("ignore")

# --- Asumsi: DataFrame train dan test sudah tersedia ---
# Contoh:
# train = pd.read_csv('data_train.csv')
# test = pd.read_csv('data_test.csv')

# Siapkan hasil submission (salinan dari test)
submission = test.copy()
submission['price'] = np.nan  # Inisialisasi kolom prediksi

# List untuk menyimpan detail MAPE tiap kombinasi (komoditas, provinsi)
mape_list = []

# Looping berdasarkan kombinasi: *komoditas* terlebih dahulu, kemudian *provinsi*
for (komoditas, provinsi), group in train.groupby(['komoditas', 'provinsi']):
    # Filter data train & test yang sesuai
    train_data = group.sort_values('Date')
    test_data = test[(test['komoditas'] == komoditas) & (test['provinsi'] == provinsi)].sort_values('Date')
    
    if len(train_data) < 30:
        print(f"Data terlalu sedikit untuk {komoditas} - {provinsi}, skip...")
        continue
    
    n = len(train_data)
    split_idx = int(n * 0.8)
    if split_idx < 1 or split_idx == n:
        print(f"Data untuk {komoditas} - {provinsi} tidak cukup untuk split 80/20.")
        continue

    # --- Evaluasi MAPE menggunakan 80/20 split ---
    train_eval = train_data.iloc[:split_idx]
    val_eval   = train_data.iloc[split_idx:]
    
    try:
        model_eval = ARIMA(train_eval['harga'], order=(5, 1, 0))
        model_fit_eval = model_eval.fit()
        forecast_eval = model_fit_eval.forecast(steps=len(val_eval))
        
        # Hitung MAPE (nilai desimal, misalnya 0.1234)
        actual = val_eval['harga'].values
        pred_eval = forecast_eval.values
        mape = np.mean(np.abs((actual - pred_eval) / actual))
        
        mape_list.append({
            'komoditas': komoditas,
            'provinsi': provinsi,
            'n_data': n,
            'mape': mape
        })
        print(f"[80/20] {komoditas} - {provinsi}: n={n}, MAPE={mape:.4f}")
    except Exception as e:
        print(f"Gagal evaluasi {komoditas} - {provinsi}: {e}")
        continue

    # --- Pelatihan model final menggunakan 100% data ---
    try:
        model_final = ARIMA(train_data['harga'], order=(5, 1, 0))
        model_fit_final = model_final.fit()
        forecast_final = model_fit_final.forecast(steps=len(test_data))
        
        # Masukkan hasil prediksi ke submission
        submission.loc[test_data.index, 'price'] = forecast_final.values
    except Exception as e:
        print(f"Gagal proses final {komoditas} - {provinsi}: {e}")
        continue

# Hapus kolom yang tidak diperlukan pada submission
submission = submission.drop(columns=['provinsi', 'komoditas', 'Date'])

# Simpan file submission ke CSV
submission.to_csv('final_arima.csv', index=False)
print("Prediksi selesai! File 'final.csv' telah dibuat 🚀")

# --- Simpan hasil MAPE ke file Excel ---
df_mape = pd.DataFrame(mape_list)
if not df_mape.empty:
    overall_avg_mape = df_mape['mape'].mean()
else:
    overall_avg_mape = None

df_overall = pd.DataFrame({
    'overall_avg_mape': [overall_avg_mape],
    'n_models': [len(df_mape)]
})

with pd.ExcelWriter('mape_results_final_arima.xlsx') as writer:
    df_mape.to_excel(writer, sheet_name='MAPE_Detail', index=False)
    df_overall.to_excel(writer, sheet_name='MAPE_Overall', index=False)

print("Hasil MAPE telah disimpan ke 'mape_results_final_arima.xlsx'.")

[80/20] Bawang Merah - Aceh: n=1004, MAPE=0.2767
[80/20] Bawang Merah - Bali: n=1004, MAPE=0.2087
[80/20] Bawang Merah - Banten: n=1004, MAPE=0.2242
[80/20] Bawang Merah - Bengkulu: n=1004, MAPE=0.2769
[80/20] Bawang Merah - DI Yogyakarta: n=1004, MAPE=0.3071
[80/20] Bawang Merah - DKI Jakarta: n=1004, MAPE=0.2135
[80/20] Bawang Merah - Gorontalo: n=1004, MAPE=0.2090
[80/20] Bawang Merah - Jambi: n=1004, MAPE=0.2550
[80/20] Bawang Merah - Jawa Barat: n=1004, MAPE=0.2276
[80/20] Bawang Merah - Jawa Tengah: n=1004, MAPE=0.2638
[80/20] Bawang Merah - Jawa Timur: n=1004, MAPE=0.2732
[80/20] Bawang Merah - Kalimantan Barat: n=1004, MAPE=0.1484
[80/20] Bawang Merah - Kalimantan Selatan: n=1004, MAPE=0.2199
[80/20] Bawang Merah - Kalimantan Tengah: n=1004, MAPE=0.1999
[80/20] Bawang Merah - Kalimantan Timur: n=1004, MAPE=0.1615
[80/20] Bawang Merah - Kalimantan Utara: n=1004, MAPE=0.1596
[80/20] Bawang Merah - Kepulauan Bangka Belitung: n=1004, MAPE=0.2526
[80/20] Bawang Merah - Kepulauan Ria

In [35]:

import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.metrics import mean_absolute_percentage_error
import warnings

# Abaikan warning yang tidak perlu
warnings.filterwarnings("ignore")

# --- Asumsi: DataFrame train dan test sudah tersedia ---
# Contoh:
# train = pd.read_csv('data_train.csv')
# test = pd.read_csv('data_test.csv')

# Siapkan hasil submission (salinan dari test)
submission = test.copy()
submission['price'] = np.nan  # Inisialisasi kolom prediksi

# List untuk menyimpan detail MAPE tiap kombinasi (komoditas, provinsi)
mape_list = []

# Looping berdasarkan kombinasi: *komoditas* terlebih dahulu, kemudian *provinsi*
for (komoditas, provinsi), group in train.groupby(['komoditas', 'provinsi']):
    # Filter data train & test yang sesuai
    train_data = group.sort_values('Date')
    test_data = test[(test['komoditas'] == komoditas) & (test['provinsi'] == provinsi)].sort_values('Date')
    
    if len(train_data) < 30:
        print(f"Data terlalu sedikit untuk {komoditas} - {provinsi}, skip...")
        continue
    
    n = len(train_data)
    split_idx = int(n * 0.8)
    if split_idx < 1 or split_idx == n:
        print(f"Data untuk {komoditas} - {provinsi} tidak cukup untuk split 80/20.")
        continue

    # --- Evaluasi MAPE menggunakan 80/20 split ---
    train_eval = train_data.iloc[:split_idx]
    val_eval   = train_data.iloc[split_idx:]
    
    try:
        # Feature & Target
        X_train = np.arange(len(train_eval)).reshape(-1, 1)
        y_train = train_eval['harga'].values
        X_val = np.arange(len(train_eval), len(train_eval) + len(val_eval)).reshape(-1, 1)
        y_val = val_eval['harga'].values
        
        # Train model
        model_eval = xgb.XGBRegressor(n_estimators=200, learning_rate=0.05, max_depth=5, random_state=42)
        model_eval.fit(X_train, y_train)
        forecast_eval = model_eval.predict(X_val)
        
        # Hitung MAPE
        mape = mean_absolute_percentage_error(y_val, forecast_eval)
        mape_list.append({'komoditas': komoditas, 'provinsi': provinsi, 'n_data': n, 'mape': mape})
        print(f"[80/20] {komoditas} - {provinsi}: n={n}, MAPE={mape:.4f}")
    except Exception as e:
        print(f"Gagal evaluasi {komoditas} - {provinsi}: {e}")
        continue

    # --- Pelatihan model final menggunakan 100% data ---
    try:
        X_full = np.arange(len(train_data)).reshape(-1, 1)
        y_full = train_data['harga'].values
        X_test = np.arange(len(train_data), len(train_data) + len(test_data)).reshape(-1, 1)
        
        model_final = xgb.XGBRegressor(n_estimators=200, learning_rate=0.05, max_depth=5, random_state=42)
        model_final.fit(X_full, y_full)
        forecast_final = model_final.predict(X_test)
        
        # Masukkan hasil prediksi ke submission
        submission.loc[test_data.index, 'price'] = forecast_final
    except Exception as e:
        print(f"Gagal proses final {komoditas} - {provinsi}: {e}")
        continue

# Hapus kolom yang tidak diperlukan pada submission
submission = submission.drop(columns=['provinsi', 'komoditas', 'Date'])

# Simpan file submission ke CSV
submission.to_csv('final_xgb.csv', index=False)
print("Prediksi selesai! File 'final_xgb.csv' telah dibuat 🚀")

# --- Simpan hasil MAPE ke file Excel ---
df_mape = pd.DataFrame(mape_list)
if not df_mape.empty:
    overall_avg_mape = df_mape['mape'].mean()
else:
    overall_avg_mape = None

df_overall = pd.DataFrame({'overall_avg_mape': [overall_avg_mape], 'n_models': [len(df_mape)]})

with pd.ExcelWriter('mape_results_final_xgb.xlsx') as writer:
    df_mape.to_excel(writer, sheet_name='MAPE_Detail', index=False)
    df_overall.to_excel(writer, sheet_name='MAPE_Overall', index=False)

print("Hasil MAPE telah disimpan ke 'mape_results_final_xgb.xlsx'.")

[80/20] Bawang Merah - Aceh: n=1004, MAPE=0.2752
[80/20] Bawang Merah - Bali: n=1004, MAPE=0.2121
[80/20] Bawang Merah - Banten: n=1004, MAPE=0.2242
[80/20] Bawang Merah - Bengkulu: n=1004, MAPE=0.2748
[80/20] Bawang Merah - DI Yogyakarta: n=1004, MAPE=0.3116
[80/20] Bawang Merah - DKI Jakarta: n=1004, MAPE=0.2154
[80/20] Bawang Merah - Gorontalo: n=1004, MAPE=0.2088
[80/20] Bawang Merah - Jambi: n=1004, MAPE=0.2574
[80/20] Bawang Merah - Jawa Barat: n=1004, MAPE=0.2255
[80/20] Bawang Merah - Jawa Tengah: n=1004, MAPE=0.2587
[80/20] Bawang Merah - Jawa Timur: n=1004, MAPE=0.2725
[80/20] Bawang Merah - Kalimantan Barat: n=1004, MAPE=0.1490
[80/20] Bawang Merah - Kalimantan Selatan: n=1004, MAPE=0.2192
[80/20] Bawang Merah - Kalimantan Tengah: n=1004, MAPE=0.1993
[80/20] Bawang Merah - Kalimantan Timur: n=1004, MAPE=0.1616
[80/20] Bawang Merah - Kalimantan Utara: n=1004, MAPE=0.1596
[80/20] Bawang Merah - Kepulauan Bangka Belitung: n=1004, MAPE=0.2506
[80/20] Bawang Merah - Kepulauan Ria

In [36]:

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_percentage_error
import warnings

# Abaikan warning yang tidak perlu
warnings.filterwarnings("ignore")

# --- Asumsi: DataFrame train dan test sudah tersedia ---
# Contoh:
# train = pd.read_csv('data_train.csv')
# test = pd.read_csv('data_test.csv')

# Siapkan hasil submission (salinan dari test)
submission = test.copy()
submission['price'] = np.nan  # Inisialisasi kolom prediksi

# List untuk menyimpan detail MAPE tiap kombinasi (komoditas, provinsi)
mape_list = []

# Looping berdasarkan kombinasi: *komoditas* terlebih dahulu, kemudian *provinsi*
for (komoditas, provinsi), group in train.groupby(['komoditas', 'provinsi']):
    # Filter data train & test yang sesuai
    train_data = group.sort_values('Date')
    test_data = test[(test['komoditas'] == komoditas) & (test['provinsi'] == provinsi)].sort_values('Date')
    
    if len(train_data) < 30:
        print(f"Data terlalu sedikit untuk {komoditas} - {provinsi}, skip...")
        continue
    
    n = len(train_data)
    split_idx = int(n * 0.8)
    if split_idx < 1 or split_idx == n:
        print(f"Data untuk {komoditas} - {provinsi} tidak cukup untuk split 80/20.")
        continue

    # --- Evaluasi MAPE menggunakan 80/20 split ---
    train_eval = train_data.iloc[:split_idx]
    val_eval   = train_data.iloc[split_idx:]
    
    try:
        # Feature & Target
        X_train = np.arange(len(train_eval)).reshape(-1, 1)
        y_train = train_eval['harga'].values
        X_val = np.arange(len(train_eval), len(train_eval) + len(val_eval)).reshape(-1, 1)
        y_val = val_eval['harga'].values
        
        # Train model
        model_eval = RandomForestRegressor(n_estimators=100, random_state=42)
        model_eval.fit(X_train, y_train)
        forecast_eval = model_eval.predict(X_val)
        
        # Hitung MAPE
        mape = mean_absolute_percentage_error(y_val, forecast_eval)
        mape_list.append({'komoditas': komoditas, 'provinsi': provinsi, 'n_data': n, 'mape': mape})
        print(f"[80/20] {komoditas} - {provinsi}: n={n}, MAPE={mape:.4f}")
    except Exception as e:
        print(f"Gagal evaluasi {komoditas} - {provinsi}: {e}")
        continue

    # --- Pelatihan model final menggunakan 100% data ---
    try:
        X_full = np.arange(len(train_data)).reshape(-1, 1)
        y_full = train_data['harga'].values
        X_test = np.arange(len(train_data), len(train_data) + len(test_data)).reshape(-1, 1)
        
        model_final = RandomForestRegressor(n_estimators=100, random_state=42)
        model_final.fit(X_full, y_full)
        forecast_final = model_final.predict(X_test)
        
        # Masukkan hasil prediksi ke submission
        submission.loc[test_data.index, 'price'] = forecast_final
    except Exception as e:
        print(f"Gagal proses final {komoditas} - {provinsi}: {e}")
        continue

# Hapus kolom yang tidak diperlukan pada submission
submission = submission.drop(columns=['provinsi', 'komoditas', 'Date'])

# Simpan file submission ke CSV
submission.to_csv('final_rf.csv', index=False)
print("Prediksi selesai! File 'final_rf.csv' telah dibuat 🚀")

# --- Simpan hasil MAPE ke file Excel ---
df_mape = pd.DataFrame(mape_list)
if not df_mape.empty:
    overall_avg_mape = df_mape['mape'].mean()
else:
    overall_avg_mape = None

df_overall = pd.DataFrame({'overall_avg_mape': [overall_avg_mape], 'n_models': [len(df_mape)]})

with pd.ExcelWriter('mape_results_final_rf.xlsx') as writer:
    df_mape.to_excel(writer, sheet_name='MAPE_Detail', index=False)
    df_overall.to_excel(writer, sheet_name='MAPE_Overall', index=False)

print("Hasil MAPE telah disimpan ke 'mape_results_final_rf.xlsx'.")

[80/20] Bawang Merah - Aceh: n=1004, MAPE=0.2756
[80/20] Bawang Merah - Bali: n=1004, MAPE=0.2111
[80/20] Bawang Merah - Banten: n=1004, MAPE=0.2242
[80/20] Bawang Merah - Bengkulu: n=1004, MAPE=0.2757
[80/20] Bawang Merah - DI Yogyakarta: n=1004, MAPE=0.3093
[80/20] Bawang Merah - DKI Jakarta: n=1004, MAPE=0.2137
[80/20] Bawang Merah - Gorontalo: n=1004, MAPE=0.2088
[80/20] Bawang Merah - Jambi: n=1004, MAPE=0.2565
[80/20] Bawang Merah - Jawa Barat: n=1004, MAPE=0.2258
[80/20] Bawang Merah - Jawa Tengah: n=1004, MAPE=0.2600
[80/20] Bawang Merah - Jawa Timur: n=1004, MAPE=0.2729
[80/20] Bawang Merah - Kalimantan Barat: n=1004, MAPE=0.1488
[80/20] Bawang Merah - Kalimantan Selatan: n=1004, MAPE=0.2194
[80/20] Bawang Merah - Kalimantan Tengah: n=1004, MAPE=0.1996
[80/20] Bawang Merah - Kalimantan Timur: n=1004, MAPE=0.1614
[80/20] Bawang Merah - Kalimantan Utara: n=1004, MAPE=0.1596
[80/20] Bawang Merah - Kepulauan Bangka Belitung: n=1004, MAPE=0.2518
[80/20] Bawang Merah - Kepulauan Ria

In [37]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
import warnings
import itertools
import os

warnings.filterwarnings("ignore")

# --- Asumsi: DataFrame train dan test sudah tersedia ---
# Contoh:
# train = pd.read_csv('data_train.csv')
# test = pd.read_csv('data_test.csv')

submission = test.copy()
submission['price'] = np.nan

mape_list = []

# Fungsi untuk mencari parameter SARIMA terbaik (dipangkas agar lebih cepat)
def sarimax_grid_search(train_series, seasonal_period=12):
    p = d = q = range(0, 2)  # Kurangi rentang
    P = D = Q = range(0, 1)  # Kurangi rentang
    s = [seasonal_period]
    
    best_aic = float("inf")
    best_order = None
    best_seasonal_order = None

    for order in itertools.product(p, d, q):
        for seasonal_order in itertools.product(P, D, Q, s):
            try:
                model = SARIMAX(train_series, order=order, seasonal_order=seasonal_order,
                                enforce_stationarity=False, enforce_invertibility=False)
                result = model.fit(disp=False)
                
                if result.aic < best_aic:
                    best_aic = result.aic
                    best_order = order
                    best_seasonal_order = seasonal_order
            except:
                continue
    
    return best_order, best_seasonal_order

# Loop berdasarkan kombinasi komoditas dan provinsi
for (komoditas, provinsi), group in train.groupby(['komoditas', 'provinsi']):
    train_data = group.sort_values('Date')
    test_data = test[(test['komoditas'] == komoditas) & (test['provinsi'] == provinsi)].sort_values('Date')
    
    if len(train_data) < 30:
        print(f"Data terlalu sedikit untuk {komoditas} - {provinsi}, skip...")
        continue
    
    n = len(train_data)
    split_idx = int(n * 0.8)
    if split_idx < 1 or split_idx == n:
        print(f"Data untuk {komoditas} - {provinsi} tidak cukup untuk split 80/20.")
        continue
    
    train_eval = train_data.iloc[:split_idx]
    val_eval = train_data.iloc[split_idx:]
    
    try:
        # Gunakan grid search dengan parameter lebih kecil
        best_order, best_seasonal_order = sarimax_grid_search(train_eval['harga'])

        # Model evaluasi
        model_eval = SARIMAX(train_eval['harga'], order=best_order, seasonal_order=best_seasonal_order,
                             enforce_stationarity=False, enforce_invertibility=False)
        model_fit_eval = model_eval.fit(disp=False)
        forecast_eval = model_fit_eval.forecast(steps=len(val_eval))
        
        actual = val_eval['harga'].values
        pred_eval = forecast_eval.values
        mape = np.mean(np.abs((actual - pred_eval) / actual))
        
        mape_list.append({
            'komoditas': komoditas,
            'provinsi': provinsi,
            'n_data': n,
            'mape': mape
        })
        print(f"[80/20] {komoditas} - {provinsi}: n={n}, MAPE={mape:.4f}")
    except Exception as e:
        print(f"Gagal evaluasi {komoditas} - {provinsi}: {e}")
        continue
    
    try:
        # Model final
        model_final = SARIMAX(train_data['harga'], order=best_order, seasonal_order=best_seasonal_order,
                              enforce_stationarity=False, enforce_invertibility=False)
        model_fit_final = model_final.fit(disp=False)
        forecast_final = model_fit_final.forecast(steps=len(test_data))
        
        submission.loc[test_data.index, 'price'] = forecast_final.values
    except Exception as e:
        print(f"Gagal proses final {komoditas} - {provinsi}: {e}")
        continue

submission = submission.drop(columns=['provinsi', 'komoditas', 'Date'])
submission.to_csv('final_sarimax_fast.csv', index=False)
print("Prediksi selesai! File 'final_sarimax_fast.csv' telah dibuat 🚀")

# Simpan hasil MAPE ke file Excel
df_mape = pd.DataFrame(mape_list)
if not df_mape.empty:
    overall_avg_mape = df_mape['mape'].mean()
else:
    overall_avg_mape = None

df_overall = pd.DataFrame({
    'overall_avg_mape': [overall_avg_mape],
    'n_models': [len(df_mape)]
})

with pd.ExcelWriter('mape_results_final_sarimax.xlsx') as writer:
    df_mape.to_excel(writer, sheet_name='MAPE_Detail', index=False)
    df_overall.to_excel(writer, sheet_name='MAPE_Overall', index=False)

print("Hasil MAPE telah disimpan ke 'mape_results_final_sarimax_fast.xlsx'.")


[80/20] Bawang Merah - Aceh: n=1004, MAPE=0.2994
[80/20] Bawang Merah - Bali: n=1004, MAPE=0.2086
[80/20] Bawang Merah - Banten: n=1004, MAPE=0.2242
[80/20] Bawang Merah - Bengkulu: n=1004, MAPE=0.2795
[80/20] Bawang Merah - DI Yogyakarta: n=1004, MAPE=0.3083
[80/20] Bawang Merah - DKI Jakarta: n=1004, MAPE=0.2136
[80/20] Bawang Merah - Gorontalo: n=1004, MAPE=0.2089
[80/20] Bawang Merah - Jambi: n=1004, MAPE=0.2565
[80/20] Bawang Merah - Jawa Barat: n=1004, MAPE=0.2276
[80/20] Bawang Merah - Jawa Tengah: n=1004, MAPE=0.2654
[80/20] Bawang Merah - Jawa Timur: n=1004, MAPE=0.2733
[80/20] Bawang Merah - Kalimantan Barat: n=1004, MAPE=0.1483
[80/20] Bawang Merah - Kalimantan Selatan: n=1004, MAPE=0.2193
[80/20] Bawang Merah - Kalimantan Tengah: n=1004, MAPE=0.2004
[80/20] Bawang Merah - Kalimantan Timur: n=1004, MAPE=0.1614
[80/20] Bawang Merah - Kalimantan Utara: n=1004, MAPE=0.1596
[80/20] Bawang Merah - Kepulauan Bangka Belitung: n=1004, MAPE=0.2523
[80/20] Bawang Merah - Kepulauan Ria

# SUBMISSION

In [39]:

import pandas as pd

# Load submission files
sub1 = pd.read_csv(r"D:\Data Sains\dataset\comodity-price-prediction-penyisihan-arkavidia-9\iris n\New\final_arima.csv")
sub2 = pd.read_csv(r"D:\Data Sains\dataset\comodity-price-prediction-penyisihan-arkavidia-9\iris n\New\final_sarimax_fast.csv")
sub3 = pd.read_csv(r"D:\Data Sains\dataset\comodity-price-prediction-penyisihan-arkavidia-9\iris n\New\final_xgb.csv")
sub4 = pd.read_csv(r"D:\Data Sains\dataset\comodity-price-prediction-penyisihan-arkavidia-9\iris n\New\final_rf.csv")

# Pastikan semua file memiliki kolom yang sama
assert sub1.columns.equals(sub2.columns) and sub1.columns.equals(sub3.columns) and sub1.columns.equals(sub4.columns), "Kolom submission tidak cocok!"

# Cek jumlah NaN sebelum ensemble
print(f"NaN sebelum ensemble:\n{sub1.isna().sum()}\n")

# Gabungkan hasil dengan rata-rata (mengabaikan NaN)
submission_final = sub1.copy()
submission_final['price'] = pd.concat([sub1['price'], sub2['price'], sub3['price'], sub4['price']], axis=1).mean(axis=1, skipna=True)

# Tangani nilai NaN setelah ensemble
if submission_final['price'].isna().sum() > 0:
    print("Terdapat NaN setelah ensemble, melakukan imputasi...")
    submission_final['price'] = submission_final['price'].fillna(method='ffill')  # Bisa juga .fillna(0) atau .fillna(submission_final['price'].median())

# Simpan hasil
submission_final.to_csv('submission_final_ensemble.csv', index=False)
print("FInal Ensemble submission berhasil dibuat! 🚀")

NaN sebelum ensemble:
id       0
price    0
dtype: int64

FInal Ensemble submission berhasil dibuat! 🚀
